In [1]:
%store -r summary_data_dir_activity
%store -r activity_all_file


%store -r summary_data_dir_person
%store -r persons_all_file

%store -r events_nonmoma

%store -r json_suffix

%store -r linked_data_filepath_group

%store -r datavis_venue_exhibitions 


%store -r linked_data_filepath_activity
%store -r linked_data_filepath_person

In [2]:
try:
    import json
except:
    %pip install json
    import json

from operator import itemgetter

import os

try:
    import urllib
except:
    %pip install urllib
    import urllib

try:
    import hashlib
except:
    %pip install hashlib
    import hashlib

## add image URL for person from wikidata

In [3]:
person_image_dict = {}


# open summary file api/persons_all.json
with open(os.path.join(summary_data_dir_person, persons_all_file), 'r') as json_file:
    data = json.load(json_file)

    #iterate through each person in persons_all.json
    for person in data["persons"]:

        image_url = ""
        # get id
        person_id = person.get("id")

        external_ids = person.get("equivalent")

# http://vocab.getty.edu/ulan/500006860
# http://www.wikidata.org/entity/Q19595180",
# http://viaf.org/viaf/95722748"
        
        for external_id in external_ids:

            if len(external_id.split("wikidata.org")) > 1:
                
                qid = external_id.split("/").pop()
                url = "https://www.wikidata.org/w/api.php?action=wbgetclaims&property=P18&format=json&entity=" + qid


                try:
                    response = urllib.request.urlopen(url)
                    data = json.loads(response.read())

                    image_filename = data.get('claims').get('P18')[0].get("mainsnak").get("datavalue").get("value")
                    image_filename = image_filename.replace(" ", "_")
                
                    if image_filename != "" and ".tif" not in image_filename:

                        hash = hashlib.md5(image_filename.encode('utf-8')).hexdigest()
                        a = hash[0]
                        b = hash[1]
                        image_url = "https://upload.wikimedia.org/wikipedia/commons/" + a + "/" + a + b + "/" + image_filename

                        with open(os.path.join("./data/wikidata", person_id.split("/").pop() + ".json"), 'w') as json_file:
                            json_file.write(json.dumps({"image_filename" : image_filename, "image_url" : image_url},indent=2))
                except:
                    continue



